In [8]:
import pandas as pd

# Carregando o DataFrame tratado
df = pd.read_csv('TelecomX_data_tratados.csv')

print("Colunas originais antes da remoção:")
print(df.columns)
print("\nPrimeiras 5 linhas do DataFrame original:")
print(df.head())

# Identificando e eliminando colunas que não agregam valor preditivo
# 'ID_Cliente' é um identificador único e geralmente não é útil para prever churn.
# 'Unnamed: 0' é uma coluna de índice que pode ter sido gerada ao salvar o CSV, e também não é útil.
colunas_para_eliminar = ['ID_Cliente', 'Unnamed: 0'] 

# Verificando se as colunas existem antes de tentar eliminá-las para evitar erros
colunas_existentes_para_eliminar = [col for col in colunas_para_eliminar if col in df.columns]

if colunas_existentes_para_eliminar:
    df = df.drop(columns=colunas_existentes_para_eliminar)
    print(f"\nColunas eliminadas: {colunas_existentes_para_eliminar}")
else:
    print("\nNenhuma das colunas especificadas para eliminação foi encontrada.")

print("\nColunas após a eliminação:")
print(df.columns)
print("\nPrimeiras 5 linhas do DataFrame após a eliminação:")
print(df.head())

Colunas originais antes da remoção:
Index(['ID_Cliente', 'Cancelou', 'Genero', 'Idoso', 'Possui_Conjuge',
       'Possui_Dependentes', 'Meses_Permanencia', 'Servico_Telefone',
       'Multiplas_Linhas', 'Tipo_Internet', 'OnlineSecurity', 'Backup_Online',
       'Protecao_Dispositivo', 'Suporte_Tecnico', 'Streaming_TV',
       'Streaming_Filmes', 'Tipo_Contrato', 'Fatura_Digital',
       'Metodo_Pagamento', 'Cobranca_Mensal', 'Cobranca_Total'],
      dtype='object')

Primeiras 5 linhas do DataFrame original:
   ID_Cliente Cancelou     Genero  Idoso Possui_Conjuge Possui_Dependentes  \
0  0002-ORFBO      Não   Feminino      0            Sim                Sim   
1  0003-MKNFE      Não  Masculino      0            Não                Não   
2  0004-TLHLJ      Sim  Masculino      0            Não                Não   
3  0011-IGKFF      Sim  Masculino      1            Sim                Não   
4  0013-EXCHZ      Sim   Feminino      1            Sim                Não   

   Meses_Permanenc

### Transformando váriaveis categóricas em formato numérico

In [9]:
# Identificando as colunas categóricas para o one-hot encoding
colunas_categoricas = ['Tipo_Contrato', 'Metodo_Pagamento', 'Genero']

# Realizando o One-Hot Encoding
# O parâmetro 'columns' recebe a lista de colunas a serem transformadas.
# O 'drop_first=True' é uma boa prática para evitar multicolinearidade (uma das colunas criadas é redundante).
df_encoded = pd.get_dummies(df, columns=colunas_categoricas, drop_first=True)

# A coluna 'Cancelou' já está no formato numérico (Sim=1, Não=0) de acordo com seu relatório,
# então não precisa de encoding, apenas para certificar que ela existe e foi tratada.
df_encoded['Cancelou'] = df_encoded['Cancelou'].map({'Sim': 1, 'Não': 0})
df_encoded.dropna(subset=['Cancelou'], inplace=True)

print("Colunas do DataFrame original:")
print(df.columns.tolist())
print("\nColunas do DataFrame após o One-Hot Encoding:")
print(df_encoded.columns.tolist())
print("\nPrimeiras 5 linhas do DataFrame após a transformação:")
print(df_encoded.head())

Colunas do DataFrame original:
['Cancelou', 'Genero', 'Idoso', 'Possui_Conjuge', 'Possui_Dependentes', 'Meses_Permanencia', 'Servico_Telefone', 'Multiplas_Linhas', 'Tipo_Internet', 'OnlineSecurity', 'Backup_Online', 'Protecao_Dispositivo', 'Suporte_Tecnico', 'Streaming_TV', 'Streaming_Filmes', 'Tipo_Contrato', 'Fatura_Digital', 'Metodo_Pagamento', 'Cobranca_Mensal', 'Cobranca_Total']

Colunas do DataFrame após o One-Hot Encoding:
['Cancelou', 'Idoso', 'Possui_Conjuge', 'Possui_Dependentes', 'Meses_Permanencia', 'Servico_Telefone', 'Multiplas_Linhas', 'Tipo_Internet', 'OnlineSecurity', 'Backup_Online', 'Protecao_Dispositivo', 'Suporte_Tecnico', 'Streaming_TV', 'Streaming_Filmes', 'Fatura_Digital', 'Cobranca_Mensal', 'Cobranca_Total', 'Tipo_Contrato_Mês a mês', 'Tipo_Contrato_Um ano', 'Metodo_Pagamento_Cheque eletrônico', 'Metodo_Pagamento_Cheque por correio', 'Metodo_Pagamento_Transferência bancária (automática)', 'Genero_Masculino']

Primeiras 5 linhas do DataFrame após a transformação

### Verificando a proporção de evasão

In [10]:
# Calculando a contagem de clientes para cada classe (Evadiram e Ativos)
contagem_classes = df['Cancelou'].value_counts()

# Calculando a proporção em porcentagem
proporcao_classes = df['Cancelou'].value_counts(normalize=True) * 100

print("Contagem de clientes por classe:")
print(f"Ativos (0): {contagem_classes[0]} clientes")
print(f"Evadiram (1): {contagem_classes[1]} clientes")

print("\nProporção das classes:")
print(f"Ativos (0): {proporcao_classes[0]:.2f}%")
print(f"Evadiram (1): {proporcao_classes[1]:.2f}%")

# Avaliando o desequilíbrio de classes
if proporcao_classes[1] < 30:
    print("\nAVISO: O desequilíbrio de classes é significativo.")
    print("A classe minoritária (Evadiram) representa menos de 30% do total de clientes.")
    print("Isso pode afetar o desempenho dos modelos preditivos, fazendo-os tender a prever a classe majoritária.")
    print("Considere usar técnicas de reamostragem, como oversampling ou undersampling, para balancear as classes durante o treinamento do modelo.")
else:
    print("\nAs classes estão relativamente balanceadas.")

Contagem de clientes por classe:
Ativos (0): 5174 clientes
Evadiram (1): 1869 clientes

Proporção das classes:
Ativos (0): 73.46%
Evadiram (1): 26.54%

AVISO: O desequilíbrio de classes é significativo.
A classe minoritária (Evadiram) representa menos de 30% do total de clientes.
Isso pode afetar o desempenho dos modelos preditivos, fazendo-os tender a prever a classe majoritária.
Considere usar técnicas de reamostragem, como oversampling ou undersampling, para balancear as classes durante o treinamento do modelo.


C:\Users\farney.colab\AppData\Local\Temp\ipykernel_2396\3776942201.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Ativos (0): {contagem_classes[0]} clientes")
C:\Users\farney.colab\AppData\Local\Temp\ipykernel_2396\3776942201.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Evadiram (1): {contagem_classes[1]} clientes")
C:\Users\farney.colab\AppData\Local\Temp\ipykernel_2396\3776942201.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by po

### Balanceamento de Classes 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# CORREÇÃO: Transformando a coluna 'Cancelou' para o formato numérico 0 e 1 no início
# Isso é crucial para que os algoritmos de balanceamento funcionem.
df['Cancelou'] = df['Cancelou'].replace({'Sim': 1, 'Não': 0})
df.dropna(subset=['Cancelou'], inplace=True)
df['Cancelou'] = df['Cancelou'].astype(int)

# Codificando as variáveis categóricas restantes
colunas_categoricas = ['Tipo_Contrato', 'Metodo_Pagamento', 'Genero']
# Aqui, aplicamos o get_dummies em todo o DataFrame, exceto na coluna 'Cancelou'
df_encoded = pd.get_dummies(df, columns=colunas_categoricas, drop_first=True)

# Separando as variáveis preditoras (X) da variável alvo (y)
X = df_encoded.drop('Cancelou', axis=1)
y = df_encoded['Cancelou']

# --- 2. Divisão dos Dados em Treino e Teste ---

# O balanceamento deve ser feito APENAS nos dados de treino.
# Usamos 'stratify=y' para manter a mesma proporção de classes nos conjuntos de treino e teste.
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print("Distribuição das classes ANTES do balanceamento (apenas nos dados de treino):")
print(f"Classe 'Não Evadiu' (0): {Counter(y_treino)[0]}")
print(f"Classe 'Evadiu' (1): {Counter(y_treino)[1]}")
print("-" * 50)

# --- 3. Balanceamento com SMOTE (Oversampling) ---

# O SMOTE cria novas amostras sintéticas para a classe minoritária
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_treino, y_treino)

print("Distribuição das classes APÓS SMOTE (Oversampling):")
print(f"Classe 'Não Evadiu' (0): {Counter(y_smote)[0]}")
print(f"Classe 'Evadiu' (1): {Counter(y_smote)[1]}")
print("-" * 50)

# --- 4. Balanceamento com RandomUnderSampler (Undersampling) ---

# O Undersampling remove aleatoriamente amostras da classe majoritária
undersampler = RandomUnderSampler(random_state=42)
X_under, y_under = undersampler.fit_resample(X_treino, y_treino)

print("Distribuição das classes APÓS RandomUndersampler (Undersampling):")
print(f"Classe 'Não Evadiu' (0): {Counter(y_under)[0]}")
print(f"Classe 'Evadiu' (1): {Counter(y_under)[1]}")
print("-" * 50)